# Imports

In [134]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

from sklearn.metrics import mean_squared_error
from scipy.stats import norm


# Question 1

(1 point) Import the time series data. 

Create a time series of quarterly growth rates of GDP by taking
the first differences of the log-levels and multiply by 100. 

Denote this series by ∆yt. Are the quarterly
growth rates normally distributed?

In [3]:
# import GDPEA.csv
df = pd.read_csv('GDPEA.csv')

In [4]:
df

,DATE,GDP
0,1995-01-01,1835998.0
1,1995-04-01,1846613.1
2,1995-07-01,1853116.3
3,1995-10-01,1859777.7
4,1996-01-01,1862148.5
...,...,...
108,2022-01-01,2716755.9
109,2022-04-01,2741228.7
110,2022-07-01,2751656.0
111,2022-10-01,2750179.8


In [5]:
# take first difference of the log-levels and multiply by 100 of the column 'GDP'
df['GDP_growth'] = 100 * np.log(df['GDP']).diff()

# print the first 5 rows of the data frame
df.head()

,DATE,GDP,GDP_growth
0,1995-01-01,1835998.0,NaN
1,1995-04-01,1846613.1,0.576500
2,1995-07-01,1853116.3,0.351550
3,1995-10-01,1859777.7,0.358826
4,1996-01-01,1862148.5,0.127396


In [6]:
def plot_hist_and_jb_test(df, col_name, title):
    fig = px.histogram(df, x=col_name, nbins=50, title=f'Histogram of {title}')
    fig.show()

    jb_test = jarque_bera(df[col_name].dropna())
    print('The test statistic is: ', jb_test[0])
    print('The p-value is: ', jb_test[1])
    print('Skewness is: ', jb_test[2])
    print('Kurtosis is: ', jb_test[3])

    if jb_test[1] < 0.05:
        return 'The growth rates are not normally distributed.'
    else:
        return 'The growth rates are normally distributed.'

In [7]:
plot_hist_and_jb_test(df, 'GDP_growth', 'GDP growth rates')

The test statistic is:  6567.305750040752
The p-value is:  0.0
Skewness is:  -1.0642473408443531
Kurtosis is:  40.45329510163537


'The growth rates are not normally distributed.'

# Question 2

(2 points) Repeat question (1.), but using the sample Q1 1995 to Q3 2008 and (b) Q1 1995 to Q4 2019.

What are the most striking differences compared to (1.)? What does this tell you about the observations
in Q4 2008, Q1 2009, and in 2020? 

Also plot the time series over the full sample to argue about those
observations.

In [8]:
# Create sample from 1995-01-01 till 2008-07-01 and another sample from 1995-01-01 till 2019-10-01
df_sample1 = df[(df['DATE'] >= '1995-01-01') & (df['DATE'] <= '2008-07-01')]
df_sample2 = df[(df['DATE'] >= '1995-01-01') & (df['DATE'] <= '2019-10-01')]

In [9]:
plot_hist_and_jb_test(df_sample1, 'GDP_growth', 'GDP growth rates from 1995-01-01 till 2008-07-01')

The test statistic is:  0.779508310613518
The p-value is:  0.677223345799987
Skewness is:  -0.28563410963352354
Kurtosis is:  3.141777148038425


'The growth rates are normally distributed.'

In [10]:
plot_hist_and_jb_test(df_sample2, 'GDP_growth', 'GDP growth rates from 1995-01-01 till 2019-10-01')

The test statistic is:  1045.3700625623092
The p-value is:  1.0017864222482736e-227
Skewness is:  -2.937079164188037
Kurtosis is:  17.795854466803217


'The growth rates are not normally distributed.'

In [11]:
# plot the obeservation of 2008-10-01, 2009-01-01, 2020-01-01, 2020-04-01, 2020-07-01, 2020-10-01
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['DATE'], y=df['GDP_growth'], mode='lines', name='GDP growth'))

obs_dates = ['2008-10-01', '2009-01-01', '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01']
obs_values = df.loc[df['DATE'].isin(obs_dates), 'GDP_growth']

fig.add_trace(go.Scatter(x=obs_dates,
                         y=obs_values,
                         mode='markers',
                         name='Outliers',
                         text=obs_dates,
                         textposition='top center'))

fig.update_layout(title='GDP growth rates with outliers of 2008-10-01, 2009-01-01, 2020-01-01, 2020-04-01, 2020-07-01, 2020-10-01',
                    xaxis_title='Date',
                    yaxis_title='GDP growth rates')

fig.show()

# Question 3

(3 points) Estimate an AR(p) model for ∆yt selecting the lag length based on AIC (with a maximal
lag length of 8). 

Evaluate the model by inspecting the properties of the residuals by applying tests for
normality and no-autocorrelation. 

What kind of patterns do you observe in the residuals around Q1 2020
to Q4 2020?

In [12]:
diff_y = df['GDP_growth'].values[1:]

In [13]:
def find_optimal_lag(y, max_lag=8):
    """
    Finds the optimal lag length using AIC and plots the AIC values.
    
    Parameters:
    y (array-like): The time series data.
    max_lag (int): The maximum lag length to consider.
    
    Returns:
    int: The optimal lag length.
    """
    aic_values = []
    for p in range(1, max_lag+1):
        model = ARIMA(endog=y, order=(p, 0, 0))
        model_fit = model.fit()
        aic_values.append(model_fit.aic)

    print('AIC values for lag length 1 to {}: {}'.format(max_lag, aic_values))

    # Plot the AIC values
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, max_lag+1)), y=aic_values, mode='lines+markers'))
    fig.update_layout(title='AIC values for lag length 1 to {}'.format(max_lag),
                        xaxis_title='Lag length',
                        yaxis_title='AIC values')
    # make the minimum value red
    fig.add_trace(go.Scatter(x=[aic_values.index(min(aic_values))+1], y=[min(aic_values)], mode='markers', marker_color='red'))
    fig.show()
    
    return aic_values.index(min(aic_values)) + 1

In [14]:
best_lag = find_optimal_lag(diff_y)
print('The optimal lag length is: ', best_lag)

AIC values for lag length 1 to 8: [441.7112290925104, 441.8546445573909, 443.670416484652, 445.52167024476245, 446.808919215419, 448.08363418405656, 449.9634665877808, 451.5846148050765]


The optimal lag length is:  1


In [15]:
def plot_residuals(y, best_lag):
    """
    Plots the residuals of an ARIMA model with a specified lag.
    
    Parameters:
    y (array-like): The time series data.
    best_lag (int): The optimal lag length.
    """
    model = ARIMA(endog=y, order=(best_lag, 0, 0))
    model_fit = model.fit()
    residuals = model_fit.resid

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['DATE'].values[best_lag:], y=residuals, mode='lines+markers'))
    fig.update_layout(title='Residuals of the AR({}) model'.format(best_lag),
                        xaxis_title='Date',
                        yaxis_title='Residuals')
    fig.show()

    return residuals

In [16]:
residuals = plot_residuals(diff_y, best_lag)

Note, comment about the outliers of 2020-Q1 till 2020-Q4

In [17]:
# plot the histogram of the residuals
plot_hist_and_jb_test(pd.DataFrame(residuals), 0, 'residuals of the AR(1) model')

The test statistic is:  7076.422018358543
The p-value is:  0.0
Skewness is:  -3.540914520668044
Kurtosis is:  41.29130246179157


'The growth rates are not normally distributed.'

In [18]:
dw = durbin_watson(residuals)
print('The Durbin-Watson test statistic is: ', dw)
# Interpret the test statistic
if dw < 1.5:
    print('Positive autocorrelation')
elif dw > 2.5:
    print('Negative autocorrelation')
else:
    print('No autocorrelation')

The Durbin-Watson test statistic is:  2.0624496738369174
No autocorrelation


In general, if d is less than 1.5 or greater than 2.5 then there is potentially a serious autocorrelation problem. 

Otherwise, if d is between 1.5 and 2.5 then autocorrelation is likely not a cause for concern.

# Question 4

(2 points) Test whether the observations in Q4 2008, Q1 2009, Q1 2020, Q2 2020, and Q3 2020 are
innovation outliers. 

What do you conclude from the test results?

-> Approach:

To test whether the observations in Q4 2008, Q1 2009, Q1 2020, Q2 2020, and Q3 2020 are innovation outliers, 

you can use statistical methods such as the Z-score or modified Z-score.

In [19]:
# Create a matrix of zeros for the dummy variables
z = np.zeros((len(df), 5))

# Set the values of the dummy variables for the outliers
z[df['DATE'] == '2008-10-01', 0] = 1
z[df['DATE'] == '2009-01-01', 1] = 1
z[df['DATE'] == '2020-01-01', 2] = 1
z[df['DATE'] == '2020-04-01', 3] = 1
z[df['DATE'] == '2020-07-01', 4] = 1

# Add the dummy variables to the DataFrame
df['outlier_1'] = z[:, 0]
df['outlier_2'] = z[:, 1]
df['outlier_3'] = z[:, 2]
df['outlier_4'] = z[:, 3]
df['outlier_5'] = z[:, 4]

In [20]:
# Set up the regression model
X = df[['outlier_1', 'outlier_2', 'outlier_3', 'outlier_4', 'outlier_5']]
y = df['GDP_growth']
X = sm.add_constant(X)

# drop first row
X = X.iloc[1:, :]
y = y.iloc[1:]

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             GDP_growth   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     316.6
Date:                Wed, 31 May 2023   Prob (F-statistic):           5.21e-62
Time:                        10:56:41   Log-Likelihood:                -65.856
No. Observations:                 112   AIC:                             143.7
Df Residuals:                     106   BIC:                             160.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4617      0.043     10.666      0.0

# Question 5

(2 points) Repeat question (3.) but treating the observations Q1 2009, Q1 2020, Q2 2020 and Q3 2020 as
outliers. 

Estimate an AR(2) model treating these observations as innovation outliers. 

Evaluate the model
by inspecting the properties of the residuals testing for normality and no-autocorrelation. 

Compare the
properties of the residuals to those obtained in question (3.), also comparing the plotted residuals.

In [21]:
# remove outliers from data
df_no_outliers = df[(df['DATE'] != '2009-01-01') & (df['DATE'] != '2020-01-01') & (df['DATE'] != '2020-04-01') & (df['DATE'] != '2020-07-01')]
df_no_outliers = df_no_outliers.reset_index(drop=True)

In [22]:
y_diff_no_outliers = df_no_outliers['GDP_growth'].values[1:]
best_lag_no_outliers = find_optimal_lag(y_diff_no_outliers)

AIC values for lag length 1 to 8: [129.8292632373391, 131.78287037317193, 133.3257358223754, 135.29942304612922, 136.1042365478785, 137.94273032237106, 139.7333442687439, 138.62586732094064]


In [23]:
residuals_no_outliers = plot_residuals(y_diff_no_outliers, best_lag_no_outliers)

In [38]:
# plot the histogram of the residuals
plot_hist_and_jb_test(pd.DataFrame(residuals_no_outliers), 0, 'residuals of the AR(1) model without outliers')

The test statistic is:  69.30209756321887
The p-value is:  8.938007649001005e-16
Skewness is:  0.024003867662763124
Kurtosis is:  6.924049105516986


'The growth rates are not normally distributed.'

# Question 6

Compare the in-sample fit of 

the previously estimated AR(1) model 

and 

the estimated AR(1)
model treating the observations Q1 2009, Q1 2020, Q2 2020, and Q3 2020 as innovation outliers 

based on
the value of the log likelihood and the AIC. 

Based on those in-sample fit measures which model would
you select?

In [25]:
# fit the model
model = ARIMA(endog=diff_y, order=(1, 0, 0))
model = model.fit()

# print the summary
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  112
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -217.856
Date:                Wed, 31 May 2023   AIC                            441.711
Time:                        10:56:44   BIC                            449.867
Sample:                             0   HQIC                           445.020
                                - 112                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3615      0.280      1.291      0.197      -0.187       0.910
ar.L1         -0.2283      0.043     -5.334      0.000      -0.312      -0.144
sigma2         2.8631      0.126     22.790      0.0

In [26]:
# fit the model
model_no_outliers = ARIMA(endog=y_diff_no_outliers, order=(best_lag_no_outliers, 0, 0))
model_fit_no_outliers = model_no_outliers.fit()

# print the summary
print(model_fit_no_outliers.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  108
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 -61.915
Date:                Wed, 31 May 2023   AIC                            129.829
Time:                        10:56:45   BIC                            137.876
Sample:                             0   HQIC                           133.092
                                - 108                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4383      0.084      5.224      0.000       0.274       0.603
ar.L1          0.4982      0.055      9.105      0.000       0.391       0.605
sigma2         0.1838      0.015     12.364      0.0

In [27]:
print('Log likelihood of the model with outliers: ', np.round(model.llf, 2))
print('Log likelihood of the model without outliers: ', np.round(model_fit_no_outliers.llf, 2))

print('AIC of the model with outliers: ', np.round(model.aic, 2))
print('AIC of the model without outliers: ', np.round(model_fit_no_outliers.aic, 2))

Log likelihood of the model with outliers:  -217.86
Log likelihood of the model without outliers:  -61.91
AIC of the model with outliers:  441.71
AIC of the model without outliers:  129.83


In [28]:
print('The model without outliers is better because it has a higher log likelihood and a lower AIC.')

The model without outliers is better because it has a higher log likelihood and a lower AIC.


# Question 7

(2 points) The CSV file IEA contains quarterly observations on investment in growth rates for the Euro
Area over the period Q1 1995 until Q4 2022. 

We denote the series by xt. 

Import the time series and plot
the data together with ∆yt. 

What do you observe? What is the correlation between the two series?

In [29]:
# Import IEA.csv
df_xt = pd.read_csv('IEA.csv')
df_xt

,DATE,IN
0,1995-01-01,1.12130
1,1995-04-01,1.05847
2,1995-07-01,1.04738
3,1995-10-01,1.77690
4,1996-01-01,-0.92144
...,...,...
107,2021-10-01,6.65805
108,2022-01-01,-2.84848
109,2022-04-01,9.57580
110,2022-07-01,-3.75747


In [30]:
# Plot the column 'IN' of df_xt together with the column 'GDP_growth' of df
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['DATE'], y=df['GDP_growth'], mode='lines+markers', name='GDP growth'))
fig.add_trace(go.Scatter(x=df_xt['DATE'], y=df_xt['IN'], mode='lines+markers', name='Investment in growth rates for the Euro Area'))
fig.update_layout(title='GDP growth and investment in growth rates for the Euro Area',
                    xaxis_title='Date',
                    yaxis_title='Value')
fig.show()

In [31]:
print('We observe potential high correlation between the two time series.')

We observe potential high correlation between the two time series.


In [32]:
# correlation between GDP growth and investment
print('The correlation between GDP growth and investment is: ', np.round(df['GDP_growth'].corr(df_xt['IN']), 2))

The correlation between GDP growth and investment is:  0.82


# Question 8

(3 points) Estimate an AR(p) model for xt selecting the lag length based on AIC 
(with a maximal lag
length of 8). 

Next, estimate an AR(p) model for xt including ∆yt as an additional regressor. 

Select the
lag length of the model based on AIC (with a maximal lag length of 8). 

Evaluate the two models in the
usual way, by considering the properties of the residuals. 

Is the second model an improvement over the
AR(p) model for xt?

In [34]:
best_lag_xt = find_optimal_lag(df_xt['IN'])
print('The optimal lag length is: ', best_lag_xt)

AIC values for lag length 1 to 8: [629.4240892723839, 630.5945169877093, 631.7569242723324, 633.7156938480323, 635.7110202413614, 636.0311384939985, 637.5624475369168, 629.4991103632967]


The optimal lag length is:  1


In [47]:
# estimate an AR(1) model for xt including diff_y as an additional regressor. 
# Use the optimal lag length for xt.

model = ARIMA(endog=df_xt['IN'], order=(best_lag_xt, 0, 0))
model_fit = model.fit()

# print the summary
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                     IN   No. Observations:                  112
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -311.712
Date:                Wed, 31 May 2023   AIC                            629.424
Time:                        11:32:27   BIC                            637.580
Sample:                             0   HQIC                           632.733
                                - 112                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5845      0.478      1.223      0.221      -0.352       1.521
ar.L1          0.0186      0.039      0.483      0.629      -0.057       0.094
sigma2        15.3084      0.767     19.961      0.0

In [48]:
# Investigate residuals
residuals_xt = model_fit.resid

# plot the residuals
plot_residuals(residuals_xt, best_lag_xt)

0      0.536725
1      0.462948
2      0.453157
3      1.182901
4     -1.530317
         ...   
107    6.144864
108   -3.557026
109    9.061430
110   -4.525514
111    1.752122
Length: 112, dtype: float64

In [49]:
plot_hist_and_jb_test(pd.DataFrame(residuals_xt), 0, 'residuals of the AR(1) model for xt')

The test statistic is:  1002.4830150645665
The p-value is:  2.058634173678443e-218
Skewness is:  0.03379395001643368
Kurtosis is:  17.65650779749993


'The growth rates are not normally distributed.'

In [50]:
# mean and variance of the residuals
print('The mean of the residuals is: ', np.round(np.mean(residuals_xt), 2))
print('The variance of the residuals is: ', np.round(np.var(residuals_xt), 2))

The mean of the residuals is:  0.0
The variance of the residuals is:  15.31


In [51]:
# estimate an AR(1) model for xt including diff_y as an additional regressor. 
# Use the optimal lag length for xt.

model = ARIMA(endog=df_xt['IN'], order=(best_lag_xt, 0, 0), exog=diff_y)
model_fit = model.fit()

# print the summary
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                     IN   No. Observations:                  112
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -309.787
Date:                Wed, 31 May 2023   AIC                            627.573
Time:                        11:32:37   BIC                            638.447
Sample:                             0   HQIC                           631.985
                                - 112                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7808      0.636      1.227      0.220      -0.466       2.028
x1            -0.5315      0.066     -8.050      0.000      -0.661      -0.402
ar.L1          0.1877      0.055      3.427      0.0

In [52]:
# Investigate residuals
residuals_xt = model_fit.resid

# plot the residuals
plot_residuals(residuals_xt, best_lag_xt)

0      0.647673
1      0.329404
2      0.363190
3      0.970451
4     -1.556008
         ...   
107    6.905829
108   -4.466240
109    9.685468
110   -6.468564
111    2.500992
Length: 112, dtype: float64

In [53]:
plot_hist_and_jb_test(pd.DataFrame(residuals_xt), 0, 'residuals of the AR(1) model for xt with diff_y as an additional regressor')

The test statistic is:  1030.245625095097
The p-value is:  1.92754714079322e-224
Skewness is:  1.1721521714277205
Kurtosis is:  17.672121755605225


'The growth rates are not normally distributed.'

In [54]:
# mean and variance of the residuals
print('The mean of the residuals is: ', np.round(np.mean(residuals_xt), 2))
print('The variance of the residuals is: ', np.round(np.var(residuals_xt), 2))

The mean of the residuals is:  -0.0
The variance of the residuals is:  14.79


-> Evaluate the two models in the
usual way, by considering the properties of the residuals. 

Is the second model an improvement over the
AR(p) model for xt?

-> Second model slightly better:
For residuals, slightly lower variance and also higher loglikelihood and lower AIC

# Question 9

(3 points) Estimate a threshold model for xt with an AR(1) model in both regimes, with ∆yt as the
threshold variable, and with the threshold fixed at zero. 

Discuss the differences between the AR(1)
models in the two regimes based on the estimated coefficients. 

Also, explain the interpretation of setting
the threshold fixed at zero for the specified threshold variable.

In [58]:
data = pd.DataFrame({'xt': df_xt['IN'], 'diff_y': diff_y})
data['threshold'] = np.where(data['diff_y'] >= 0, 1, 0)
data


,xt,diff_y,threshold
0,1.12130,0.576500,1
1,1.05847,0.351550,1
2,1.04738,0.358826,1
3,1.77690,0.127396,1
4,-0.92144,0.690434,1
...,...,...,...
107,6.65805,0.590516,1
108,-2.84848,0.896777,1
109,9.57580,0.379666,1
110,-3.75747,-0.053662,0


In [62]:
# Split the data into two regimes based on the threshold variable
regime_1 = data[data['threshold'] == 0]['xt']
regime_2 = data[data['threshold'] == 1]['xt']

# Estimate the ARIMA model in regime 1
model_1 = ARIMA(regime_1, order=(1, 0, 0))
result_1 = model_1.fit()

# Estimate the ARIMA model in regime 2
model_2 = ARIMA(regime_2, order=(1, 0, 0))
result_2 = model_2.fit()

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-package

In [65]:
# Print the model summary for both regimes
print("Regime 1:")
print(result_1.summary())

Regime 1:
                               SARIMAX Results                                
Dep. Variable:                     xt   No. Observations:                   19
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 -64.071
Date:                Wed, 31 May 2023   AIC                            134.142
Time:                        12:14:56   BIC                            136.975
Sample:                             0   HQIC                           134.621
                                 - 19                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0606      2.342     -0.026      0.979      -4.652       4.530
ar.L1          0.1120      0.211      0.532      0.595      -0.301       0.525
sigma2        49.6881     12.162      4.08

In [66]:
print("\nRegime 2:")
print(result_2.summary())


Regime 2:
                               SARIMAX Results                                
Dep. Variable:                     xt   No. Observations:                   93
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -228.972
Date:                Wed, 31 May 2023   AIC                            463.945
Time:                        12:15:02   BIC                            471.542
Sample:                             0   HQIC                           467.012
                                 - 93                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7099      0.415      1.712      0.087      -0.103       1.523
ar.L1         -0.0167      0.157     -0.106      0.916      -0.325       0.292
sigma2         8.0547      0.429     18.7

# Question 10

(2 points) Estimate the same threshold model but now with the threshold as an unknown parameter
(instead of fixing it at 0). 

Discuss the differences between the results for the threshold model with fixed
threshold at zero and the threshold model with unknown threshold.

In [78]:
# Specify the model with a threshold as an unknown parameter
model = MarkovRegression(data['xt'], k_regimes=2, exog=data[['diff_y']])
threshold_estimate = 0.0  # Initial estimate for the threshold

# Iteratively estimate the threshold
for i in range(10):  # Repeat the estimation procedure several times for convergence
    model_fit = model.fit(em_iter=i, threshold=threshold_estimate)
    threshold_estimate = np.mean(model_fit.smoothed_marginal_probabilities[1])  # Estimate threshold from regime probabilities

# Print the final model summary
print(model_fit.summary())

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/base/optimizer.py:18: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: threshold. After release 0.14, this will raise.



                        Markov Switching Model Results                        
Dep. Variable:                     xt   No. Observations:                  112
Model:               MarkovRegression   Log Likelihood                -267.516
Date:                Wed, 31 May 2023   AIC                            549.032
Time:                        12:56:49   BIC                            568.062
Sample:                             0   HQIC                           556.753
                                - 112                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.240      0.922      0.357      -0.249       0.692
x1             0.7431      0.165      4.494      0.0

In [79]:
# print final threshold estimate
print('The final threshold estimate is: ', np.round(threshold_estimate, 2))

The final threshold estimate is:  0.03


-> No need to mention the final threshold, just discuss the results and differences

# Question 11

(2 points) Estimate a logistic smooth transition model (including an intercept and one lag of xt and using
∆yt as threshold variable). 

Estimate the smoothing parameter (γ) and threshold parameter (c). 

Discuss
the differences between the results for this model and the threshold model with unknown threshold.

In [81]:
# Add intercept and lag of xt to the data
data['intercept'] = 1  # Add intercept
data['lag_xt'] = data['xt'].shift(1)  # Add one lag of xt

# Remove missing values introduced by lagging
data = data.dropna()

# Estimate the logistic smooth transition model
model = MarkovRegression(data['xt'], exog=data[['intercept', 'lag_xt']], k_regimes=2)
result = model.fit()

# Print the estimated parameters
print("Intercept:", result.params[0])
print("Coefficient for lag_xt:", result.params[1])

# Print the model summary
print(result.summary())

Intercept: 0.9256708568659534
Coefficient for lag_xt: 0.4029635007867309
                        Markov Switching Model Results                        
Dep. Variable:                     xt   No. Observations:                  111
Model:               MarkovRegression   Log Likelihood                -278.625
Date:                Wed, 31 May 2023   AIC                            575.250
Time:                        13:14:31   BIC                            599.636
Sample:                             0   HQIC                           585.143
                                - 111                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0731   4.16e+04  -1.76e-06      1.000   -

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



-> Do not forget to discuss

the differences between the results for this model and the threshold model with unknown threshold.

# Question 12

(2 points) Use the SupF approach to test for threshold nonlinearity in the threshold model, but with the
threshold as an unknown parameter (instead of fixing it at 0). 

Discuss the test results.

1. Estimate the threshold model with the threshold as an unknown parameter, as described earlier.

2. Obtain the residuals from the estimated threshold model.

3. Estimate two auxiliary regression models, one for each regime, using the residuals and a set of non-linear functions of the threshold variable.

4. Compute the sum of squared residuals for each auxiliary regression.

5. Perform an F-test to compare the sum of squared residuals between the two auxiliary regression models. The null hypothesis is that there is no threshold nonlinearity, and the alternative hypothesis is that there is threshold nonlinearity.

In [101]:
# Specify the model with a threshold as an unknown parameter
model = MarkovRegression(data['xt'], k_regimes=2, exog=data[['diff_y']])
threshold_estimate = 0.0  # Initial estimate for the threshold

# Iteratively estimate the threshold
for i in range(10):  # Repeat the estimation procedure several times for convergence
    model_fit = model.fit(em_iter=i, threshold=threshold_estimate)
    threshold_estimate = np.mean(model_fit.smoothed_marginal_probabilities[1])  # Estimate threshold from regime probabilities

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/base/optimizer.py:18: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: threshold. After release 0.14, this will raise.



In [102]:
# get residuals
residuals_threshold = model_fit.resid

In [105]:
# add residuals to data
data['residuals'] = residuals_threshold

# regime 1 residuals 
regime_1_res = data[data['threshold'] == 0]['residuals']

# regime 2 residuals
regime_2_res = data[data['threshold'] == 1]['residuals']

/var/folders/df/ss33svv50gq79h_yy76phkq80000gn/T/ipykernel_17434/3333232989.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
# Estimate two auxiliary regression models, one for each regime, using the residuals and a set of non-linear functions of the threshold variable.
# The non-linear functions are the following

# Estimate the auxiliary regression model for regime 1
model_1 = ARIMA(regime_1_res, order=(1, 0, 0))
result_1 = model_1.fit()

# Estimate the auxiliary regression model for regime 2
model_2 = ARIMA(regime_2_res, order=(1, 0, 0))
result_2 = model_2.fit()

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-package

In [109]:
print("Regime 1:")
print(result_1.summary())


Regime 1:
                               SARIMAX Results                                
Dep. Variable:              residuals   No. Observations:                   19
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 -54.070
Date:                Wed, 31 May 2023   AIC                            114.140
Time:                        13:30:20   BIC                            116.974
Sample:                             0   HQIC                           114.620
                                 - 19                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8989      1.315     -0.683      0.494      -3.477       1.679
ar.L1          0.1024      0.284      0.361      0.718      -0.454       0.658
sigma2        17.3425      5.332      3.25

In [110]:
print("\nRegime 2:")
print(result_2.summary())


Regime 2:
                               SARIMAX Results                                
Dep. Variable:              residuals   No. Observations:                   92
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -177.035
Date:                Wed, 31 May 2023   AIC                            360.070
Time:                        13:30:20   BIC                            367.636
Sample:                             0   HQIC                           363.124
                                 - 92                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1799      0.157      1.143      0.253      -0.128       0.488
ar.L1         -0.1064      0.082     -1.293      0.196      -0.268       0.055
sigma2         2.7472      0.331      8.3

In [117]:
# Compute sum of squared residuals
ssr_1 = np.sum(result_1.resid ** 2)
ssr_2 = np.sum(result_2.resid ** 2)

# Perform F-test
F_statistic = ((ssr_2 - ssr_1) / 1) / (ssr_1 / (len(residuals) - 2))

# Print the F-statistic and p-value
print("F-statistic:", F_statistic)

F-statistic: -25.39261416909439


In [120]:
# p-value for the F-statistic
from scipy.stats import f
print("p-value:", f.sf(F_statistic, 1, len(residuals) - 2))

p-value: 1.0


# Question 13

(4 points) Use the threshold model with unknown threshold and the logistic smooth transition model
to obtain fitted values. 

Evaluate the relative accuracy of these in-sample forecasts based on mean
squared prediction errors. 

Which of the nonlinear models delivers more accurate forecasts? 

Are potential
improvements in forecast accuracy statistically significant? Discuss also the limitation of comparing the
models only based on in-sample fit.


In [131]:
# Use the threshold model with unknown threshold and the logistic smooth transition model to obtain fitted values. 
# Plot the fitted values and the actual values of xt.

# Specify the model with a threshold as an unknown parameter
model = MarkovRegression(data['xt'], k_regimes=2, exog=data[['diff_y']])
threshold_estimate = 0.0  # Initial estimate for the threshold

# Iteratively estimate the threshold
for i in range(10):  # Repeat the estimation procedure several times for convergence
    model_fit = model.fit(em_iter=i, threshold=threshold_estimate)
    threshold_estimate = np.mean(model_fit.smoothed_marginal_probabilities[1])  # Estimate threshold from regime probabilities

# get fitted values
fitted_values = model_fit.fittedvalues

# Evaluate the relative accuracy of these in-sample forecasts based on mean squared prediction errors. 
# Print the mean squared prediction error for the threshold model with unknown threshold and the logistic smooth transition model.
mse_m1 = mean_squared_error(data['xt'], fitted_values)
print('Mean squared prediction error:', np.round(mse_m1, 2))

# plot fitted values and actual values using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['xt'], mode='lines', name='actual'))
fig.add_trace(go.Scatter(x=data.index, y=fitted_values, mode='lines', name='fitted'))
fig.update_layout(title='Fitted values using the threshold model with unknown threshold vs actual values of xt', xaxis_title='Date', yaxis_title='xt')
fig.show()

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/base/optimizer.py:18: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: threshold. After release 0.14, this will raise.



Mean squared prediction error: 5.47


In [132]:
# Estimate the logistic smooth transition model
model = MarkovRegression(data['xt'], exog=data[['intercept', 'lag_xt']], k_regimes=2)
result = model.fit()

# get fitted values
fitted_values = result.fittedvalues

# Evaluate the relative accuracy of these in-sample forecasts based on mean squared prediction errors.
# Print the mean squared prediction error for the threshold model with unknown threshold and the logistic smooth transition model.
mse_m2 = mean_squared_error(data['xt'], fitted_values)
print('Mean squared prediction error:', np.round(mse_m2, 2))

# plot fitted values and actual values using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['xt'], mode='lines', name='actual'))
fig.add_trace(go.Scatter(x=data.index, y=fitted_values, mode='lines', name='fitted'))
fig.update_layout(title='Fitted values using logistic smooth transition model vs actual values of xt', xaxis_title='Date', yaxis_title='xt')
fig.show()


Mean squared prediction error: 6.84


/Users/sunny/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



-> Which of the nonlinear models delivers more accurate forecasts? 

In [135]:
# Are potential improvements in forecast accuracy statistically significant? 
# Perform a Diebold-Mariano test to answer this question.

dm_statistic = (mse_m1 - mse_m2) / np.sqrt((mse_m1 ** 2 + mse_m2 ** 2) / 2)

# Print the DM test statistic and the p-value
print("DM test statistic:", dm_statistic)
p_value = 2 * (1 - norm.cdf(np.abs(dm_statistic)))
print("p-value:", p_value)


# Interpret the results of the Diebold-Mariano test.
if p_value < 0.05:
    print("The logistic smooth transition model is significantly better than the threshold model with unknown threshold.")
else:
    print("The logistic smooth transition model is not significantly better than the threshold model with unknown threshold.")

DM test statistic: -0.22141383513999452
p-value: 0.8247702208627714
The logistic smooth transition model is not significantly better than the threshold model with unknown threshold.


-> Discuss also the limitation of comparing the
models only based on in-sample fit.

One of the main limitations is that it may not accurately reflect the model's ability to forecast out-of-sample data. A model that fits the in-sample data well may not necessarily perform well when used to make predictions on new data.

Another limitation is that in-sample fit can be affected by overfitting, which occurs when a model is too complex and fits the noise in the data rather than the underlying signal. This can lead to poor out-of-sample performance and inaccurate predictions.